based on https://www.kaggle.com/the1owl/fractals-of-nature-blend-0-90050

In [16]:
from multiprocessing import Pool, cpu_count
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import fbeta_score
from PIL import Image, ImageStat
from skimage import io
import xgboost as xgb
import pandas as pd
import numpy as np
import glob, cv2
import random
import scipy
import os
import bcolz
from tqdm import tqdm_notebook

In [6]:


def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()


def load_array(fname):
    return bcolz.open(fname)[:]

def f2_score(y_true, y_pred):
    # fbeta_score throws a confusing error if inputs are not numpy arrays
    y_true, y_pred, = np.array(y_true), np.array(y_pred)
    # We need to use average='samples' here, any other average method will generate bogus results
    return fbeta_score(y_true, y_pred, beta=2, average='samples')

    

In [7]:
def get_features(path):
    try:
        st = []
        #pillow jpg
        img = Image.open(path)
        im_stats_ = ImageStat.Stat(img)
        st += im_stats_.sum
        st += im_stats_.mean
        st += im_stats_.rms
        st += im_stats_.var
        st += im_stats_.stddev
        img = np.array(img)[:,:,:3]
        st += [scipy.stats.kurtosis(img[:,:,0].ravel())]
        st += [scipy.stats.kurtosis(img[:,:,1].ravel())]
        st += [scipy.stats.kurtosis(img[:,:,2].ravel())]
        st += [scipy.stats.skew(img[:,:,0].ravel())]
        st += [scipy.stats.skew(img[:,:,1].ravel())]
        st += [scipy.stats.skew(img[:,:,2].ravel())]
        #cv2 jpg
        img = cv2.imread(path)
        bw = cv2.imread(path,0)
        st += list(cv2.calcHist([bw],[0],None,[256],[0,256]).flatten()) #bw 
        st += list(cv2.calcHist([img],[0],None,[256],[0,256]).flatten()) #r
        st += list(cv2.calcHist([img],[1],None,[256],[0,256]).flatten()) #g
        st += list(cv2.calcHist([img],[2],None,[256],[0,256]).flatten()) #b
        try:
            #skimage tif
            p1 = path.replace('jpg','tif')
            p1 = p1.replace('train-tif','train-tif-v2') #Why make path changes so complex that they nullify old scripts
            p1 = p1.replace('test-tif','test-tif-v2') #Why make path changes so complex that they nullify old scripts
            imgr = io.imread(p1)
            tf = imgr[:, :, 3]
            st += list(cv2.calcHist([tf],[0],None,[256],[0,65536]).flatten()) #near ifrared
            ndvi = ((imgr[:, :, 3] - imgr[:, :, 0]) / (imgr[:, :, 3] + imgr[:, :, 0])) #water ~ -1.0, barren area ~ 0.0, shrub/grass ~ 0.2-0.4, forest ~ 1.0
            st += list(np.histogram(ndvi,bins=20, range=(-1,1))[0])
            ndvi = ((imgr[:, :, 3] - imgr[:, :, 1]) / (imgr[:, :, 3] + imgr[:, :, 1]))
            st += list(np.histogram(ndvi,bins=20, range=(-1,1))[0])
            ndvi = ((imgr[:, :, 3] - imgr[:, :, 2]) / (imgr[:, :, 3] + imgr[:, :, 2]))
            st += list(np.histogram(ndvi,bins=20, range=(-1,1))[0])
        except:
            st += [-1 for i in range(256)]
            st += [-2 for i in range(60)]
            p1 = path.replace('jpg','tif')
            p1 = p1.replace('train-tif','train-tif-v2') #Why make path changes so complex that they nullify old scripts
            p1 = p1.replace('test-tif','test-tif-v2') #Why make path changes so complex that they nullify old scripts
            print('err', p1)
        m, s = cv2.meanStdDev(img) #mean and standard deviation
        st += list(m)
        st += list(s)
        st += [cv2.Laplacian(bw, cv2.CV_64F).var()] 
        st += [cv2.Laplacian(img, cv2.CV_64F).var()]
        st += [cv2.Sobel(bw,cv2.CV_64F,1,0,ksize=5).var()]
        st += [cv2.Sobel(bw,cv2.CV_64F,0,1,ksize=5).var()]
        st += [cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5).var()]
        st += [cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5).var()]
        st += [(bw<30).sum()]
        st += [(bw>225).sum()]
    except:
        print(path)
    return [path, st]

In [8]:

def normalize_img(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_features, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    return fdata


In [9]:
in_path = 'data/'


In [46]:
y_train = []

df_train = pd.read_csv('data/train_v2.csv')
df_test = pd.read_csv('data/sample_submission_v2.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

In [47]:
for f, tags in tqdm_notebook(df_train.values, miniters=1000):
    targets = np.zeros(17)
    
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
        

    y_train.append(targets)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [48]:
y_train = np.array(y_train).astype(np.int8)

In [49]:
x_dev = pd.read_csv(in_path + 'train_v2.csv')
x_dev['path'] = x_dev['image_name'].map(lambda x: in_path + 'train-jpg/' + x + '.jpg')
# y_train = x_dev['tags'].str.get_dummies(sep=' ')





In [50]:
VALIDATION_SPLIT = 0.2

random.seed(3)
np.random.seed(3)

perm = np.random.permutation(len(x_dev))
idx_train = perm[:int(len(x_dev)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(x_dev)*(1-VALIDATION_SPLIT)):]



In [51]:
y_train[10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0], dtype=int8)

In [52]:
perm[10]

3543

In [53]:
idx_val

array([ 6658,  1687, 19366, ..., 11513,  1688,  5994])

In [54]:
X_train = x_dev.iloc[idx_train]
Y_train = y_train[idx_train]

X_valid = x_dev.iloc[idx_val]
Y_valid = y_train[idx_val]

In [55]:
# Y_train.iloc[0].values,Y_train.iloc[10].values,Y_train.iloc[200].values


In [56]:
# Y_valid.iloc[0].values,Y_valid.iloc[10].values,Y_valid.iloc[200].values

Y_valid[0],Y_valid[10],Y_valid[2000]


(array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0], dtype=int8),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int8))

In [18]:
# X_train_feats = normalize_img(X_train['path'])
# X_valid_feats = normalize_img(X_valid['path'])


In [23]:
# save_array("data/cache/X_train_feats_xgb.dat",np.array(X_train_feats))
# save_array("data/cache/X_valid_feats_xgb.dat",np.array(X_valid_feats))

In [42]:
X_train_feats = load_array("data/cache/X_train_feats_xgb.dat")
X_valid_feats = load_array("data/cache/X_valid_feats_xgb.dat")

#### test data

In [43]:
test_jpg = glob.glob(in_path + 'test-jpg/*')
test = pd.DataFrame([[p.split('/')[-1].replace('.jpg',''),p] for p in test_jpg])
test.columns = ['image_name','path']




In [25]:
# x_test_feats = normalize_img(test['path']); 
# save_array("data/cache/X_test_feats_xgb.dat",np.array(x_test_feats))

In [7]:
x_test_feats = load_array("data/cache/X_test_feats_xgb.dat")

### XGB training

In [21]:
# xgb_train = pd.DataFrame(X_train[['path']], columns=['path'])
# xgb_valid = pd.DataFrame(X_valid[['image_name']], columns=['image_name'])

In [ ]:
# for c in y.columns:
#     model = xgb.XGBClassifier(n_estimators=200, learning_rate=0.3, max_depth=4, seed=1, base_score=0.5)
#     model.fit(np.array(xtrain), y[c])
#     xgb_train[c] = model.predict_proba(np.array(xtrain))[:, 1]
#     xgb_test[c] = model.predict_proba(np.array(xtest))[:, 1]
#     print(c)

In [44]:
random_seed =1

In [28]:
val_preds = np.zeros((X_valid.shape[0], 17)).astype(np.float32)
test_preds = np.zeros((x_test_feats.shape[0], 17)).astype(np.float32)


In [36]:
Y_train.values[:, 1].reshape(-1,1).shape

(32383, 1)

In [54]:
num_folds = 5
xgb_params = {'colsample_bylevel':0.9,
 'colsample_bytree': 0.9,
 'eval_metric': 'logloss',
 'gamma': 1.,#0.02933179779163947,
 'learning_rate': 0.1,#8244194429306306,
 'max_depth': 5,
 'n_estimators': 200,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0003,#0.14600953461910307,
 'reg_lambda': 1,#1.0750571669994455,
 'scale_pos_weight': 1,
 'subsample': 0.9, #0.8097993636153147, 
             'nthread':4,'silent':1}


In [56]:
models = []

for class_i in tqdm_notebook(range(17), miniters=1): 
    
    d_train = xgb.DMatrix(X_train_feats,label= Y_train.values[:, class_i])#, weight=weight_train)
    d_valid = xgb.DMatrix(X_valid_feats,label= Y_valid.values[:,class_i])#,weight=weight_val)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]

#     model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=200, \
#                               silent=True, objective='binary:logistic', nthread=-1, \
#                               gamma=0, min_child_weight=1, max_delta_step=0, \
#                               subsample=1, colsample_bytree=1, colsample_bylevel=1, \
#                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, \
#                               base_score=0.5, seed=random_seed, missing=None)
    
#     model.fit(X_train_feats, Y_train.values[:, class_i],
#               eval_set=[(X_valid_feats,Y_valid.values[:,class_i])],
#               early_stopping_rounds=50)
    
    bst = xgb.train(xgb_params, d_train, 1000,  watchlist, early_stopping_rounds=50, verbose_eval=20)#,feval=kappa)
 


    models.append(bst)
#     test_preds[:, class_i] = model.predict_proba(x_test_feats)[:, 1]
    val_preds[:, class_i] = bst.predict(X_valid_feats)#[:, 1]
    break


[0]	train-logloss:0.641223	valid-logloss:0.642913
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.

[20]	train-logloss:0.327921	valid-logloss:0.352553
[40]	train-logloss:0.275462	valid-logloss:0.313537
[60]	train-logloss:0.255177	valid-logloss:0.303773
[80]	train-logloss:0.241647	valid-logloss:0.298754
[100]	train-logloss:0.229544	valid-logloss:0.294668
[120]	train-logloss:0.221216	valid-logloss:0.292861
[140]	train-logloss:0.214124	valid-logloss:0.291121
[160]	train-logloss:0.205627	valid-logloss:0.289588
[180]	train-logloss:0.198627	valid-logloss:0.288904
[200]	train-logloss:0.19157	valid-logloss:0.288247
[220]	train-logloss:0.18538	valid-logloss:0.287848
[240]	train-logloss:0.179043	valid-logloss:0.287279
[260]	train-logloss:0.173169	valid-logloss:0.287113
[280]	train-logloss:0.168179	valid-logloss:0.286841
[300]	train-logloss:0.16272	valid-logloss:0.286554
[320]	train-logloss:0.157

KeyboardInterrupt: 

In [ ]:
xgb_params = {'colsample_bylevel':0.9086627943049572,
 'colsample_bytree': 0.8662812861700129,
 'eval_metric': 'logloss',
 'gamma': 1.,#0.02933179779163947,
 'learning_rate': 0.07,#8244194429306306,
 'max_depth': 9,
 'n_estimators': 3500,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0003,#0.14600953461910307,
 'reg_lambda': 50,#1.0750571669994455,
 'scale_pos_weight': 1,
 'subsample': 0.8, #0.8097993636153147, 
             'nthread':16,'silent':1}

In [51]:
import pickle 

# with open('data/cache/models_xgb_v1.dump','w') as f:
#     pickle.dump(models,f,pickle.HIGHEST_PROTOCOL)

In [42]:
save_array("data/cache/xgb_tiffeatures_valpreds.dat",val_preds)

In [45]:
def optimise_f2_thresholds2(y, p, verbose=True, resolution=100,num_classes=17):
    def mf(x):
        p2 = np.zeros_like(p)
        for i in range(num_classes):
            p2[:, i] = (p[:, i] > x[i]).astype(np.int)
        score = fbeta_score(y, p2, beta=2, average='samples')
        return score

    x = [0.1]*num_classes
    for i in range(num_classes):
        best_i2 = 0
        best_score = 0
        for i2 in range(resolution):
            threshold = float(i2) / resolution
            x[i] = threshold
            score = mf(x)
            if score > best_score:
                best_i2 = threshold
                best_score = score

        x[i] = best_i2
        if verbose:
            print(i, best_i2, best_score)

    return x


In [47]:
thres = optimise_f2_thresholds2(Y_valid, val_preds,num_classes=17)

np.mean(thres)

(0, 0.23, 0.88783326343960201)
(1, 0.03, 0.88790427213724354)
(2, 0.09, 0.88791310683440661)
(3, 0.24, 0.88815583364949491)
(4, 0.09, 0.88818230174661505)
(5, 0.26, 0.88883928653794697)


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


(6, 0.07, 0.88907421041253543)
(7, 0.08, 0.88908418610297935)
(8, 0.19, 0.89107040177114327)
(9, 0.24, 0.89208064637974616)
(10, 0.19, 0.89271746682132724)
(11, 0.22, 0.89335880601464446)
(12, 0.18, 0.8938089693602661)
(13, 0.18, 0.89588005480579713)
(14, 0.18, 0.89609137014983664)
(15, 0.18, 0.89614661156096986)
(16, 0.19, 0.89849350651529125)


0.16705882352941179

In [48]:
print('F2 Score:', f2_score(Y_valid, val_preds>thres)) #combined_val_preds

('F2 Score:', 0.89849350651529125)
